In [ ]:
!pip install rawpy imageio exifread

from google.colab import files
uploaded = files.upload()

In [ ]:
# Finding defective pixel by pixel to pixel itteration (slow)

import rawpy
import numpy as np
from PIL import Image
import cv2
import io # Import io
import matplotlib.pyplot as plt

################################################################################
def load_image(file_name= 'a0151-JI2E3955.dng', sample_size = 100):
    """
    """

    with rawpy.imread(file_name) as raw:
        pattern = raw.raw_pattern
        bayer_img = raw.raw_image_visible
        color_map = {0: 'R', 1: 'G', 2: 'B', 3: 'G'}
        named_pattern = np.vectorize(color_map.get)(pattern)
        print(named_pattern)

        # select sample section of the image
        H, W = bayer_img.shape
        CY, CX = H//2, W//2
        sample = bayer_img[CY:CY+sample_size, CX:CX+sample_size]
        print("Sample dtype=", sample.dtype,";  Sample min=", np.min(sample), ";   max=", np.max(sample))

        # Convert embedded thumbnail to NumPy array
        thumbnail = raw.extract_thumb()
        print(thumbnail.format) # we this later
        if thumbnail.format == rawpy.ThumbFormat.JPEG:
            img = Image.open(io.BytesIO(thumbnail.data))
            thumbnail_np = np.array(img.convert("L"))  # convert to grayscale
        elif thumbnail.format == rawpy.ThumbFormat.BITMAP:
            thumbnail_np = np.array(thumbnail.data)
        else:
            raise ValueError("Unknown thumbnail format")

    return bayer_img, sample, thumbnail_np

################################################################################
def get_neighbors_RGGB(img, y, x, offset = 4) :

    """
    Finds the neighboring pixels for a given pixel in a Bayer image.
    """
    H, W = img.shape

    if offset == 8:
        offset_list = [(-4,0), (-2,0), (2,0), (4,0), (-2,-2), (-2,2), (2,2), (2,-2)]
    else:
        offset_list = [(-2,0), (0,-2), (2,0), (0,2)]

    neighbors = []
    for dx, dy in offset_list:
        ny, nx = y + dy, x + dx
        if 0 <= ny < H and 0 <= nx < W:
            neighbors.append(bayer_img[ny, nx])

    return neighbors

################################################################################
def DPC(input_image, threshold_factor= 4, offset = 4):
    """
    Detects and corrects defective pixels in a raw Bayer image.

    Parameters:
        bayer_img (np.ndarray): 2D Bayer image (grayscale).
        threshold (int): intensity difference threshold for detecting outliers.

    Returns:
        corrected_img (np.ndarray): image with defective pixels corrected.
        num_defective (int): number of defective pixels found and corrected.
    """
    mean_val = np.mean(input_image)
    std_val = np.std(input_image)
    threshold = int(std_val * threshold_factor)
    print("Sample dtype=", input_image.dtype,";  Sample min=", np.min(input_image), ";   max=", np.max(input_image))
    print("Pixel Mean Value= ",np.round(mean_val,0),";  Pixel Value STD=", np.round(std_val,0), ";  Threshold:", threshold)

    H, W = input_image.shape
    corrected_img = input_image.copy()
    num_defective = 0
    defective_mask = np.zeros_like(input_image, dtype=bool)

    margin = 4 if offset == 8 else 2
    for y in range(margin, H - margin):
        for x in range(margin, W - margin):
            neighbors = get_neighbors_RGGB(input_image, y, x, offset)
            medial_val = np.median(neighbors)
            if abs(input_image[y,x] - medial_val) > threshold:
                corrected_img[y, x] = int(np.mean(neighbors))
                num_defective += 1
                defective_mask[y,x] = True

    return corrected_img.astype(np.uint16), num_defective, defective_mask

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np # Import numpy
from PIL import Image # Import PIL Image
import io # Import io

bins=256

bayer_img, sample, thumbnail = load_image(file_name= 'a0151-JI2E3955.dng', sample_size= 500)

fixed_img, count, mask = DPC(sample, threshold_factor= 5.1, offset = 8)
print('Defective pixels found= ', count)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(sample, cmap='gray')
axes[0].set_title('Original Image Sample')

axes[1].imshow(mask, cmap='gray')
axes[1].set_title('DFC Mask')

axes[2].imshow(thumbnail, cmap='gray')
axes[2].set_title('Thumbnail')

for ax in axes:
    ax.axis('off')

plt.show()

In [ ]:
from skimage.filters import median
from skimage.morphology import square

# Compute median (use ksize=3 or 5 depending on your use case)
bayer_img, sample, thumbnail = load_image(file_name= 'a0151-JI2E3955.dng', sample_size= 500)
median_img = median(sample, square(3))  # works with uint16

# Compute absolute difference
diff = np.abs(sample.astype(np.int32) - median_img.astype(np.int32))

fig, axes = plt.subplots(1,2, figsize=(15, 5))
axes[0].hist(diff.ravel(), bins=512, range=(0, max), color='gray', edgecolor='black')
axes[0].set_title("Difference Histogram (|sample - median|)")
axes[0].set_xlabel("Pixel Difference")
axes[0].set_ylabel("Pixel Count")
axes[0].set_ylim(0, 1000)
axes[0].grid(True)


max = np.max(diff.ravel())
print("Max pixel difference=", max)
axes[1].hist(diff.ravel(), bins=512, range=(0, max), color='gray', edgecolor='black')
axes[1].set_title("Difference Histogram (|sample - median|)")
axes[1].set_xlabel("Pixel Difference")
axes[1].set_ylabel("Pixel Count")
axes[1].set_ylim(0, 10)
axes[1].grid(True)

In [ ]:
##  Fast defective pixel correction using OpenCV median filtering

import numpy as np
import cv2

def fast_DPC(input_image, threshold=50, ksize=3):
    """
    Fast defective pixel correction using median filtering.
    Applies a median filter and replaces outliers based on a threshold.

    Parameters:
        bayer_img (np.ndarray): 2D Bayer image.
        threshold (int): Pixel difference threshold.
        ksize (int): Median filter kernel size (must be odd).

    Returns:
        corrected_img, num_defective, defect_mask
    """
    assert ksize % 2 == 1, "ksize must be odd"

    if input_image.dtype == np.uint16:
        image_8bit = cv2.convertScaleAbs(input_image, alpha=(255.0 / 65535.0))
    else:
        image_8bit = input_image.copy()

    median_img = cv2.medianBlur(image_8bit, ksize)

    # Upscale back to original range if needed
    if input_image.dtype == np.uint16:
        median_img = (median_img.astype(np.float32) * 65535 / 255).astype(np.uint16)

    # Calculate absolute difference
    diff = np.abs(input_image.astype(np.int32) - median_img.astype(np.int32))
    defect_mask = diff > threshold

    # Correct defects
    corrected_img = input_image.copy()
    corrected_img[defect_mask] = median_img[defect_mask]

    num_defects = np.sum(defect_mask)

    return corrected_img, defect_mask, num_defects

In [ ]:
bayer_img, sample, thumbnail = load_image(file_name= 'a0151-JI2E3955.dng', sample_size= 500)

corrected_img, mask, count = fast_DPC(sample, threshold=3556, ksize=3)
print(f"Defective pixels corrected: {count}")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(sample, cmap='gray')
axes[0].set_title('Original Image Sample')

axes[1].imshow(mask, cmap='gray')
axes[1].set_title('DFC Mask')

axes[2].imshow(thumbnail, cmap='gray')
axes[2].set_title('Thumbnail')

for ax in axes:
    ax.axis('off')
plt.show()